In [18]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd 
import sys
import pickle
workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)

import allensdk.core.json_utilities as ju
#import mcmodels
from mcmodels.core import VoxelModelCache#.voxel_model_cache import VoxelModelCache
from mcmodels.core import Mask
from mcmodels.core.utils import get_structure_id, get_ordered_summary_structures
from mcmodels.utils import nonzero_unique, unionize
from mcmodels.core.utils import get_minorstructures
from mcmodels.models.crossvalidation import get_loocv_predictions
from mcmodels.core.utils import get_loss_paper
from mcmodels.core.experiment import get_voxeldata_msvd
from mcmodels.core.utils import get_loss_paper

from mcmodels.core import ModelData
#from VoxelModel import VoxelModel
#from HomogeneousModel import HomogeneousModel

#from NadarayaWatson import NadarayaWatson

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
#read data
TOP_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
INPUT_JSON = os.path.join(TOP_DIR, 'input_011520.json')
EXPERIMENTS_EXCLUDE_JSON = os.path.join(TOP_DIR, 'experiments_exclude.json')
FILE_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
OUTPUT_DIR = os.path.join(FILE_DIR, 'output')

input_data = ju.read(INPUT_JSON)
manifest_file = input_data.get('manifest_file')
manifest_file = os.path.join(TOP_DIR, manifest_file)
experiments_exclude = ju.read(EXPERIMENTS_EXCLUDE_JSON)

#its unclear why the hyperparameters are loaded from the output directory
cache = VoxelModelCache(manifest_file=manifest_file)
major_structures = input_data.get('structures')
major_structure_ids = [get_structure_id(cache, s) for s in major_structures]

#regionalize voxel model: compare with regional model
#regional parameters
cre = None
eid_set=None
high_res=False
threshold_injection = False

In [20]:
COARSE_STRUCTURE_SET_ID = 2
DEFAULT_STRUCTURE_SET_IDS = tuple([COARSE_STRUCTURE_SET_ID])
tree = cache.get_structure_tree()
default_structures = tree.get_structures_by_set_id(DEFAULT_STRUCTURE_SET_IDS)
default_structure_ids = [st['id'] for st in default_structures if st['id'] != 934]
#cre= True

In [21]:
msvds = {}
#gammas = np.asarray([0.1])
for sid in major_structure_ids:
    print(sid)
    voxel_data = ModelData(cache, sid)
    print(cre)
    experiment_ids = voxel_data.get_experiment_ids(experiments_exclude=experiments_exclude, cre=cre)
    experiment_ids = np.asarray(list(experiment_ids))    
    msvd = get_voxeldata_msvd(cache, sid,experiments_exclude,default_structure_ids,cre)
    #msvd.l2losses, msvd.paperlosses,msvd.normspredict,msvd.normtrue = single_region_cv(msvd, gammas)
    msvds[sid]  = msvd

512
None
703
None
1089
None
1097
None
315
None
313
None
354
None
698
None
771
None
803
None
477
None
549
None


In [22]:
data_info = pd.read_excel('/Users/samsonkoelle/alleninstitute/Whole Brain Cre Image Series_curation only.xlsx', 'all datasets curated_070919pull')
data_info.set_index("id", inplace=True)
ontological_order = get_ordered_summary_structures(cache)

exps = np.asarray(data_info.index.values , dtype = np.int)
creline = {}
for sid in major_structure_ids:
    msvd = msvds[sid]
    experiment_ids = np.asarray(list(msvd.experiments.keys()))
    nexp = len(experiment_ids)
    creline[sid] = np.zeros(nexp, dtype = object)
    for i in range(len(experiment_ids)):
        index = np.where(exps == experiment_ids[i])[0][0]
        creline[sid][i] = data_info['transgenic-line'].iloc[index]
        
experiments_minor_structures = {}
for sid in major_structure_ids:
    msvd = msvds[sid]
    eids = np.asarray(list(msvd.experiments.keys()))
    experiments_minor_structures[sid] = get_minorstructures(eids, data_info)

In [24]:
key = list(msvd.experiments.keys())[0]
contra_key = msvd.experiments[key].projection_mask.get_key(structure_ids=ontological_order, hemisphere_id=1)
ipsi_key = msvd.experiments[key].projection_mask.get_key(structure_ids=ontological_order, hemisphere_id=2)

cre = None
l2losses = {}
paperlosses = {}
normspredict = {}
normtrue = {}
for sid in major_structure_ids:
    #print()
    msvd = msvds[sid]
    nexp = msvd.projections.shape[0]
    ngam = 1
    
    l2losses[sid] = np.zeros((ngam,nexp))
    paperlosses[sid] = np.zeros((ngam,nexp))
    normspredict[sid] = np.zeros((ngam,nexp))
    normtrue[sid] = np.zeros(nexp)

    minor_structures = np.unique(experiments_minor_structures[sid])
    nmins = len(minor_structures)
    
    projections = msvd.projections
    ipsi_proj = unionize(projections, ipsi_key)
    contra_proj = unionize(projections, contra_key)
    reg_proj = np.hstack([ipsi_proj, contra_proj])
    msvd.reg_proj = reg_proj
    
    ipsi_target_regions, ipsi_target_counts = nonzero_unique(ipsi_key, return_counts=True)
    contra_target_regions, contra_target_counts = nonzero_unique(contra_key, return_counts=True)
    target_counts = np.concatenate([ipsi_target_counts, contra_target_counts])
    reg_proj_vcount_norm = np.divide(reg_proj, target_counts[np.newaxis, :])
    msvd.reg_proj_vcount_norm = reg_proj_vcount_norm


    source_mask = Mask.from_cache(cache, structure_ids=[sid], hemisphere_id=2)
    source_key = source_mask.get_key(structure_ids=ontological_order)  
    source_target_counts, source_target_counts =nonzero_unique(source_key, return_counts=True)
    
    injections = msvd.injections
    reg_ipsi_inj = unionize(injections, source_key)
    msvd.reg_inj = reg_ipsi_inj  
    reg_inj_vcount_norm = np.divide(reg_ipsi_inj, source_target_counts[np.newaxis, :])
    msvd.reg_inj_vcount_norm = reg_inj_vcount_norm

In [25]:
major_structures = input_data.get('structures')
major_structure_ids = [get_structure_id(cache, s) for s in major_structures]
#gammas = np.asarray([0.1,.5,1,2,10])


In [26]:
gammas = np.asarray([0.1,.5,1,2,10])

In [27]:
nms = len(major_structure_ids)
ngam = len(gammas)
wt_2ormore = {}
losses_reg_norm = np.zeros((nms,ngam))
for m in range(12):
    sid = major_structure_ids[m]
    projections = msvds[sid].reg_proj_vcount_norm
    projections = projections / np.expand_dims(np.linalg.norm(projections, axis = 1),1)
    msvds[sid].reg_proj_vcount_norm_renorm = projections
    centroids = msvds[sid].centroids
    nreg = projections.shape[1]
    nexp = projections.shape[0]
    
    nfeat = projections.shape[1]
    #ncomps = np.append(np.asarray(np.linspace(1, nfeat / 10, 5), dtype = int),nfeat - 1)
    #nalpha = len(alphas)
    
    #msvds[sid].loocv_predictions_all = np.zeros((ngam, len(ncomps),nalpha, nexp, nreg))
    #msvds[sid].loocv_predictions_wt = np.zeros((ngam, len(ncomps),nalpha, nexp, nreg))
    msvds[sid].loocv_predictions_wt = np.zeros((ngam, nexp, nreg))
    msvds[sid].loocv_predictions_all = np.zeros((ngam, nexp, nreg))
    wt_inds = np.where(creline[sid] == 'C57BL/6J')[0]
    wt_2ormore[sid] = np.asarray([])
    
    
    for g in range(ngam):
        print(g,'g')
        #for c in range(len(ncomps)):
#            print(c,'c')
            #for a in range(nalpha):
#                print(a,'a')
#         losses_reg_norm[m,g] = get_loss_paper(projections[wt_inds], loocv_predictions[wt_inds])
        msvds[sid].loocv_predictions_all[g] = get_loocv_predictions(projections, centroids, gammas[g])
        if len(wt_inds) > 1:
            msvds[sid].loocv_predictions_wt[g, wt_inds] = get_loocv_predictions(projections[wt_inds], centroids[wt_inds], gammas[g])
            if g == 0:
                wt_2ormore[sid] = np.append(wt_2ormore[sid],wt_inds)
    wt_2ormore[sid] = np.asarray(wt_2ormore[sid], dtype = int)

0 g
1 g
2 g
3 g
4 g
0 g
1 g
2 g
3 g
4 g
0 g
1 g
2 g
3 g
4 g
0 g
1 g
2 g
3 g
4 g
0 g


/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:14: RuntimeWarning: divide by zero encountered in true_divide
  weights_i = weights[i] / weights[i][otherindices].sum()
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:14: RuntimeWarning: invalid value encountered in true_divide
  weights_i = weights[i] / weights[i][otherindices].sum()
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:14: RuntimeWarning: divide by zero encountered in true_divide
  weights_i = weights[i] / weights[i][otherindices].sum()
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:14: RuntimeWarning: invalid value encountered in true_divide
  weights_i = weights[i] / weights[i][otherindices].sum()
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalida

1 g
2 g
3 g
4 g


/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:14: RuntimeWarning: divide by zero encountered in true_divide
  weights_i = weights[i] / weights[i][otherindices].sum()
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:14: RuntimeWarning: invalid value encountered in true_divide
  weights_i = weights[i] / weights[i][otherindices].sum()
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:14: RuntimeWarning: divide by zero encountered in true_divide
  weights_i = weights[i] / weights[i][otherindices].sum()
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:14: RuntimeWarning: invalid value encountered in true_divide
  weights_i = weights[i] / weights[i][otherindices].sum()
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalida

0 g
1 g
2 g
3 g
4 g
0 g
1 g
2 g
3 g
4 g
0 g
1 g
2 g
3 g
4 g
0 g
1 g
2 g
3 g
4 g
0 g
1 g
2 g
3 g
4 g
0 g
1 g
2 g
3 g
4 g
0 g
1 g
2 g
3 g
4 g


/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:14: RuntimeWarning: divide by zero encountered in true_divide
  weights_i = weights[i] / weights[i][otherindices].sum()
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:14: RuntimeWarning: invalid value encountered in true_divide
  weights_i = weights[i] / weights[i][otherindices].sum()


In [32]:
exps = np.asarray(data_info.index.values , dtype = np.int)
creline = {}
for sid in major_structure_ids:
    msvd = msvds[sid]
    experiment_ids = np.asarray(list(msvd.experiments.keys()))
    nexp = len(experiment_ids)
    creline[sid] = np.zeros(nexp, dtype = object)
    for i in range(len(experiment_ids)):
        index = np.where(exps == experiment_ids[i])[0][0]
        creline[sid][i] = data_info['transgenic-line'].iloc[index]

#inds_bad are where there is no injection
#we can use these to train but shouldnt to evaluate
#the 'wt_ind' should just be 'eval_ind'
inds_bad = {}
inds_good = {}
for sid in major_structure_ids:
    injections = msvds[sid].injections 
    inds_bad[sid] = np.where(injections.sum(axis = 1) == 0.)[0]
    inds_good[sid] = np.where(injections.sum(axis = 1) > 0.)[0]
    
nms = len(major_structure_ids)
#ngam = len(gammas)
wt_2ormore = {}
losses_reg_norm = np.zeros((nms,ngam))
inds_good_wt = {}
inds_good_wtsub = {}
for m in range(nms):
    sid = major_structure_ids[m]
    wt_inds = np.where(creline[sid] == 'C57BL/6J')[0]
    wt_2ormore[sid] = np.asarray([])
    #wt_2ormore is the indices of when there are 2 or more wild types
    if len(wt_inds) > 1:
        wt_2ormore[sid] = np.append(wt_2ormore[sid],wt_inds)
    wt_2ormore[sid] = np.asarray(wt_2ormore[sid], dtype = int)
    inds_good_wt[sid] = np.intersect1d(wt_2ormore[sid], inds_good[sid])
    inds_good_wtsub[sid] = np.where(np.isin(wt_2ormore[sid], inds_good[sid]))[0]

In [33]:
    
def get_loss(true_dict, prediction_dict,pred_ind = None, true_ind = None, keys = None):
    
    output = {}
    major_structure_ids = list(prediction_dict.keys())
    nms = len(major_structure_ids)
    ngam = prediction_dict[major_structure_ids[0]].shape[0]
    nalph = prediction_dict[major_structure_ids[0]].shape[1]
    for m in range(nms):
        sid = major_structure_ids[m]
        if pred_ind == None:
            #prediction_dict and true_dict will contain predictions for 'bad' experiments with no recorded injection
            #when we have the wild type predictions, the subset is what is good among the wild types
            #so 'true' subsetting is always good, since it is w.r.t. the full injection
            #but prediction needs good w.r.t. wt
            pind = np.asarray(list(range(prediction_dict[sid].shape[1])), dtype = int)
        else:
            pind = pred_ind[sid]
        if true_ind == None:
            tind = np.asarray(list(range(true_dict[sid].shape[0])), dtype = int)
        else:
            tind = true_ind[sid]
        
        nexp = len(pind)
        
        output[sid] = np.zeros(np.append([len(np.unique(keys[:,i])) for i in range(keys.shape[1])], nexp))

        for j in range(keys.shape[0]):
            output[sid][tuple(keys[j])] = np.asarray([get_loss_paper(true_dict[sid][tind[i]], prediction_dict[sid][tuple(keys[j])][pind[i]]) for i in range(nexp)]) 
                
    return(output)

In [34]:
homo_loocv_predictions_all = {}
homo_loocv_predictions_wt = {}
homo_reg_proj_vcount_norm_renorms= {}
for sid in major_structure_ids:
    homo_loocv_predictions_all[sid ] = msvds[sid].loocv_predictions_all
    homo_loocv_predictions_wt[sid ] = msvds[sid].loocv_predictions_wt
    homo_reg_proj_vcount_norm_renorms[sid ] = msvds[sid].reg_proj_vcount_norm_renorm

In [35]:
exps = np.asarray(data_info.index.values , dtype = np.int)
creline = {}
for sid in major_structure_ids:
    msvd = msvds[sid]
    experiment_ids = np.asarray(list(msvd.experiments.keys()))
    nexp = len(experiment_ids)
    creline[sid] = np.zeros(nexp, dtype = object)
    for i in range(len(experiment_ids)):
        index = np.where(exps == experiment_ids[i])[0][0]
        creline[sid][i] = data_info['transgenic-line'].iloc[index]

#inds_bad are where there is no injection
#we can use these to train but shouldnt to evaluate
#the 'wt_ind' should just be 'eval_ind'
inds_bad = {}
inds_good = {}
for sid in major_structure_ids:
    injections = msvds[sid].injections 
    inds_bad[sid] = np.where(injections.sum(axis = 1) == 0.)[0]
    inds_good[sid] = np.where(injections.sum(axis = 1) > 0.)[0]
    
nms = len(major_structure_ids)
#ngam = len(gammas)
wt_2ormore = {}
losses_reg_norm = np.zeros((nms,ngam))
inds_good_wt = {}
inds_good_wtsub = {}
for m in range(nms):
    sid = major_structure_ids[m]
    wt_inds = np.where(creline[sid] == 'C57BL/6J')[0]
    wt_2ormore[sid] = np.asarray([])
    #wt_2ormore is the indices of when there are 2 or more wild types
    if len(wt_inds) > 1:
        wt_2ormore[sid] = np.append(wt_2ormore[sid],wt_inds)
    wt_2ormore[sid] = np.asarray(wt_2ormore[sid], dtype = int)
    inds_good_wt[sid] = np.intersect1d(wt_2ormore[sid], inds_good[sid])
    inds_good_wtsub[sid] = np.where(np.isin(wt_2ormore[sid], inds_good[sid]))[0]

In [36]:
import itertools
a= [list(range(5))]
keys = np.asarray(list(itertools.product(*a)))

In [37]:
losses_all = get_loss(homo_reg_proj_vcount_norm_renorms, homo_loocv_predictions_all,pred_ind = inds_good, true_ind = inds_good,keys = keys)
losses_wts = get_loss(homo_reg_proj_vcount_norm_renorms, homo_loocv_predictions_wt,pred_ind = inds_good_wtsub, true_ind = inds_good_wt, keys = keys)
losses_allwts = get_loss(homo_reg_proj_vcount_norm_renorms, homo_loocv_predictions_all,pred_ind = inds_good_wt, true_ind = inds_good_wt, keys = keys)

In [39]:
def get_gamma(losses, keys):
    
    major_structure_ids = np.asarray(list(losses.keys()))
    nms = len(major_structure_ids)
    nkey = keys.shape[1]
    output = np.zeros((nms,nkey))
    for m in range(nms):
        print(m)
        sid = major_structure_ids[m]
        lvec = np.asarray([np.nanmean(losses[sid][key]) for key in keys])
        output[m] = keys[np.nanargmin(lvec)]
        #if len(np.where(np.isnan(np.nanmean(losses[sid][:,:], axis = 1)))[0]) < losses[sid].shape[0]:
        #    output[m] = np.nanargmin(np.nanmean(losses[sid][:,:], axis = 1))
    
    output = np.asarray(output, dtype = int)
    return(output)


In [40]:
import itertools
a= [list(range(5))]
keys = np.asarray(list(itertools.product(*a)))
sel_ga_all = get_gamma(losses_all,keys)
sel_ga_allwt = sel_ga_all#get_gamma(losses_allwt,keys)
#sel_ga_wt = get_gamma(losses_wts,keys)

0
1
2
3
4
5
6
7
8
9
10
11


In [41]:
def get_ms_loss(losses, gammas):
    
    major_structure_ids = np.asarray(list(losses.keys()))
    nms = len(major_structure_ids)
    output = np.zeros(nms)
    for m in range(nms):
        sid = major_structure_ids[m]
        output[m] = np.nanmean(losses[sid][gammas[m],:])
    return(output)


In [42]:
mean_nw_all = get_ms_loss(losses_all, sel_ga_all)
mean_nw_allwt = get_ms_loss(losses_allwts, sel_ga_all)
#mean_nw_wt = get_ms_loss(losses_all, sel_ga_wt)

/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  


In [43]:
mean_nw_all

array([0.90097783, 0.77737892, 0.35946378, 0.29377341, 0.43029092,
       0.47135974, 0.40976477, 0.44626504, 0.45657043, 0.53027727,
       0.69085557, 0.42870434])

In [44]:
mean_nw_allwt

array([1.20922247,        nan, 0.26124218, 0.26865134, 0.28385839,
       0.26417285, 0.39617524, 0.2177794 , 0.57632538, 0.50071204,
       0.48259818, 0.34852249])

In [45]:
losses2 = np.asarray(mean_nw_allwt[[4,7,2,1,10,9,11,3,5,8,6,0]])

In [60]:
losses2

array([0.28385839, 0.2177794 , 0.26124218,        nan, 0.48259818,
       0.50071204, 0.34852249, 0.26865134, 0.26417285, 0.57632538,
       0.39617524, 1.20922247])